## Make ASCII letter images
**Based on a matplotlib [demo](https://matplotlib.org/gallery/text_labels_and_annotations/fonts_demo_kw.html#sphx-glr-gallery-text-labels-and-annotations-fonts-demo-kw-py)**

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import random
import string
from io import BytesIO
from pathlib import Path

import imageio
import numpy as np
from skimage.color import rgb2gray
from skimage.transform import resize

In [2]:
families = ['serif', 'sans-serif', 'cursive', 'fantasy', 'monospace']
styles = ['normal', 'italic', 'oblique']
variants = ['normal', 'small-caps']
weights = ['light', 'normal', 'medium', 'semibold', 'bold', 'heavy', 'black']
sizes = ['xx-small', 'x-small', 'small', 'medium', 'large', 'x-large', 'xx-large']

In [3]:
def generate_letter(letter: str) -> np.ndarray:
    # Plot letter
    buffer = BytesIO()
    plt.figure(figsize=(10, 10))
    plt.text(
        0.5, 0.5, letter,
        family=random.choice(families),
        style=random.choice(styles),
        variant=random.choice(variants),
        weight=random.choice(weights),
        size=random.choice(sizes),
        fontsize=128,
        horizontalalignment='center',
        verticalalignment='center',
    )
    plt.axis('off')
    plt.savefig(buffer, format='png')
    plt.close()
    
    buffer.seek(0)
    img = imageio.imread(buffer)
    
    # Transform to square grayscale
    img = rgb2gray(img)
    min_i, min_j = np.argwhere(img < 1).min(axis=0)
    max_i, max_j = np.argwhere(img < 1).max(axis=0)
    img = img[min_i:max_i, min_j:max_j]
    h, w = img.shape
    height_pad = int(h * .1)
    width_pad = int(w * .1)
    img = np.pad(
        img,
        ((height_pad, height_pad), (width_pad, width_pad)),
        mode='constant', constant_values=1
    )
    img = resize(
        img, (32, 32),
        mode='constant',
        anti_aliasing=True,
    )
    return (img * 255).astype(np.uint8)

In [4]:
images = Path('letters')
images.mkdir(exist_ok=True)
for letter in string.ascii_lowercase:
    letter_images = images/f'{letter}'
    letter_images.mkdir(exist_ok=True)
    for i in range(100):
        if random.uniform(0, 1) < 0.5:
            img = generate_letter(letter)
        else:
            img = generate_letter(letter.upper())
        path = str(letter_images/f'{i:02d}.jpg')
        imageio.imwrite(path, img)